In [48]:
from ortools.sat.python import cp_model

# Case Assignment

[Link](https://dmcommunity.org/challenge/challenge-april-2025/) to the original post.

An analytical firm assigns different cases to its analysts using the following rules:

- Case must be assigned to an analyst who has the same focus area as the case type
- Analysts can not work on a new case with an amount higher than their maximum allowed case amount.
- Analysts can not work on a new case if it puts them over their maximum total cases dollar amount
- Analyst Levels must correspond to Case Complexity: analysts can work only on new cases with complexity between their Minimum Case Complexity and Maximum Case Complexity (inclusive) as described in the following table:

![](https://dmcommunity.org/wp-content/uploads/2025/04/image-3.png)

Here is the list of analysts with their current load:

![](https://dmcommunity.org/wp-content/uploads/2025/04/image-8.png)

Here is the list of new cases:

![](https://dmcommunity.org/wp-content/uploads/2025/04/image-5.png)

The firm needs to decide which analysts should be assigned to these cases. If there are multiple options, the firm prefers to minimize the overqualification when analysts are assigned to the cases below their levels. Can you create a decision service capable of addressing this and similar problems?

In [49]:
qual_level = {
1: {"Min Case Complexity": 1, "Max Case Complexity": 2},
2: {"Min Case Complexity": 1, "Max Case Complexity": 2},
3: {"Min Case Complexity": 1, "Max Case Complexity": 2},
4: {"Min Case Complexity": 1, "Max Case Complexity": 3},
5: {"Min Case Complexity": 2, "Max Case Complexity": 3},
6: {"Min Case Complexity": 2, "Max Case Complexity": 4},
7: {"Min Case Complexity": 2, "Max Case Complexity": 4},
8: {"Min Case Complexity": 3, "Max Case Complexity": 4},
9: {"Min Case Complexity": 3, "Max Case Complexity": 5},
10: {"Min Case Complexity": 4, "Max Case Complexity": 5}
}

analyst = {
"Tim Smith": {"Analyst Level": 10, "Total $ Cases Assigned": 35, "Total # Cases Assigned": 12, "Focus Area": ["Technology", "Research", "Construction"], "Max $ Allowable Cases": 50, "Total $ Cases Amount": 75},
"Sue Rogers": {"Analyst Level": 5, "Total $ Cases Assigned": 5, "Total # Cases Assigned": 10, "Focus Area": ["Technology", "Research"], "Max $ Allowable Cases": 1, "Total $ Cases Amount": 7},
"Sam Howard": {"Analyst Level": 8, "Total $ Cases Assigned": 19, "Total # Cases Assigned": 9, "Focus Area": ["Research", "Construction"], "Max $ Allowable Cases": 20, "Total $ Cases Amount": 20},
"Jill Ryan": {"Analyst Level": 9, "Total $ Cases Assigned": 14.7, "Total # Cases Assigned": 10, "Focus Area": ["Technology", "Research", "Construction"], "Max $ Allowable Cases": 1.5, "Total $ Cases Amount": 25},
"Debbie Smith": {"Analyst Level": 6, "Total $ Cases Assigned": 8, "Total # Cases Assigned": 14, "Focus Area": ["Technology", "Research"], "Max $ Allowable Cases": 10, "Total $ Cases Amount": 10},
"Debbie Bowers": {"Analyst Level": 7, "Total $ Cases Assigned": 6, "Total # Cases Assigned": 8, "Focus Area": ["Technology", "Research"], "Max $ Allowable Cases": 1, "Total $ Cases Amount": 7.5},
"Kevin Jones": {"Analyst Level": 4, "Total $ Cases Assigned": 2.8, "Total # Cases Assigned": 8, "Focus Area": ["Technology", "Research"], "Max $ Allowable Cases": 3, "Total $ Cases Amount": 3},
"Roger Howland": {"Analyst Level": 2, "Total $ Cases Assigned": .85, "Total # Cases Assigned": 7, "Focus Area": ["Construction", "Research"], "Max $ Allowable Cases": .3, "Total $ Cases Amount": 1}
}

cases = {
112: {"Case Amount": .05, "Case Complexity": 3, "Case Type": "Technology"},
113: {"Case Amount": .2, "Case Complexity": 1, "Case Type": "Technology"},
114: {"Case Amount": 1.5, "Case Complexity": 4, "Case Type": "Construction"},
115: {"Case Amount": .3, "Case Complexity": 4, "Case Type": "Research"}
}

In [50]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

u = {(i,j): model.NewBoolVar(f'choice_{i}') for i in analyst.keys() for j in cases.keys()}

for j in cases:
    # All cases must be assigned to 1 analyst
    model.AddExactlyOne([u[i,j] for i in analyst.keys()])

    for i in analyst:
        # All analysts must be assigned to at most 1 case in this list
        model.AddAtMostOne([u[i,k] for k in cases.keys()])

        # Cases assigned to analyst with same focus area
        model.Add(u[i,j] == 0).OnlyEnforceIf(cases[j]["Case Type"] not in analyst[i]["Focus Area"])

        # Analysts can not work on a new case with an amount higher than their maximum allowed case amount
        model.Add(u[i,j] == 0).OnlyEnforceIf(cases[j]["Case Amount"] > (analyst[i]["Max $ Allowable Cases"]))

        # Analysts can not work on a new case if it puts them over their maximum total cases dollar amount
        model.Add(u[i,j] == 0).OnlyEnforceIf(cases[j]["Case Amount"] > 
                                             (analyst[i]["Total $ Cases Amount"] - analyst[i]["Total $ Cases Assigned"]))
        
        # Analyst Levels must correspond to Case Complexity: analysts can work only on new cases with complexity between 
        # their Minimum Case Complexity and Maximum Case Complexity (inclusive)
        model.Add(cases[j]["Case Complexity"] >= qual_level[analyst[i]["Analyst Level"]]["Min Case Complexity"]).OnlyEnforceIf(u[i,j])
        model.Add(cases[j]["Case Complexity"] <= qual_level[analyst[i]["Analyst Level"]]["Max Case Complexity"]).OnlyEnforceIf(u[i,j])

# Minimize overqualification
model.Minimize(sum(u[i,j]*(analyst[i]["Analyst Level"] - cases[j]["Case Complexity"]) for i in analyst for j in cases))

In [51]:
status = solver.Solve(model)
print(f'Status = {solver.StatusName(status)}')
[(i,j) for i,j in u if solver.value(u[i,j]) == 1]

Status = OPTIMAL


[('Sue Rogers', 112),
 ('Jill Ryan', 114),
 ('Debbie Smith', 115),
 ('Kevin Jones', 113)]